In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
from peft import PeftModel
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model_path = "./my-llama-3.1-8b-local"
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        dtype=torch.float32,        
        device_map="cpu",                 
        low_cpu_mem_usage=True,          
        trust_remote_code=True
    )

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
lora_adapter_path = "./marketing_lora_finetuned"
os.path.exists(lora_adapter_path)

True

In [9]:
model = PeftModel.from_pretrained(base_model, lora_adapter_path)
model.eval()
print("✅ LoRA adapters loaded")

The 8-bit optimizer is not available on your device, only available on CUDA for now.


✅ LoRA adapters loaded


In [10]:
def test_marketing_model_cpu(prompt):
    """Test marketing model on CPU"""
    messages = [
        {"role": "system", "content": "You are a marketing expert who creates engaging marketing content."},
        {"role": "user", "content": prompt}
    ]
    
    try:
        formatted_prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = tokenizer(formatted_prompt, return_tensors="pt")
        
        print("🔄 Generating (this will take 2-5 minutes on CPU)...")
      
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,        
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_beams=1,            
                early_stopping=True
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        if "assistant\n" in response:
            assistant_response = response.split("assistant\n")[-1]
        else:
            assistant_response = response
        
        return assistant_response.strip()
        
    except Exception as e:
        return f"Generation failed: {e}"

In [11]:
test_prompt = "Write a short marketing message about AI tools"

print(f"\nTesting with: {test_prompt}")
print("-" * 50)

result = test_marketing_model_cpu(test_prompt)
print("✅ Generated Content:")
print(result)

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Testing with: Write a short marketing message about AI tools
--------------------------------------------------
🔄 Generating (this will take 2-5 minutes on CPU)...
✅ Generated Content:
Generation failed: probability tensor contains either `inf`, `nan` or element < 0
